In [ ]:
DEBUG = True

In [ ]:
import os
import gc
import json
import cv2
import time
import shutil
from PIL import Image
import pandas as pd
import numpy as np
import pydicom
import glob
from pydicom.pixel_data_handlers.util import apply_voi_lut
import matplotlib.pyplot as plt
import albumentations as A
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
import sys 
import timm
from timm.models.efficientnet import *
import scipy 
from scipy import stats
from albumentations.pytorch import ToTensorV2
from ast import literal_eval
import sklearn 
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold 
from sklearn.metrics import accuracy_score, roc_auc_score, average_precision_score, precision_score 
import torch
import torch.nn as nn
import torch.optim as optim 
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import GradScaler, autocast 
from torch.autograd import Variable
from torch.nn.modules.loss import _WeightedLoss
import typing as tp
try:
    from itertools import  ifilterfalse
except ImportError: # py3k
    from itertools import  filterfalse
import warnings 
if not DEBUG:
    warnings.filterwarnings('ignore')
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
    print('GPU is available')
else:
    DEVICE = torch.device('cpu')
    print('CPU is used')

In [ ]:
KAGGLE = False
MDLS_FOLDS = {'vpt3': [0, 1, 2, 3, 4]}
MDLS_FOLDS_TWOCLS = {'vptbin0': [0, 1, 2, 3, 4]}
if KAGGLE:
    DATA_PATH = '../input/siim-covid19-detection'
    MDLS_PATHS = {ver: f'../input/siim-tfmodels-{ver}' 
                  for ver, _ in MDLS_FOLDS.items()}
    MDLS_PATHS_TWOCLS = {ver: f'../input/siim-tfmodels-{ver}' 
                         for ver, _ in MDLS_FOLDS_TWOCLS.items()}
else:
    DATA_PATH = './data'
    MDLS_PATHS = {ver: f'./models_{ver}' 
                  for ver, _ in MDLS_FOLDS.items()}
    MDLS_PATHS_TWOCLS = {ver: f'./models_{ver}' 
                         for ver, _ in MDLS_FOLDS_TWOCLS.items()}
CACHE_PATH = './cache'
BATCH_SUBM = 64
WORKERS = 8
IMG_SIZE = 512
TTAS = [0, 1, 2, 3, 4, 5]

start_time = time.time()

# Cache images

In [ ]:
def read_xray(path, voi_lut=True, fix_monochrome=True):
    dicom = pydicom.read_file(path)
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array        
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8) 
    return data

def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    img = Image.fromarray(array)
    if keep_ratio:
        img.thumbnail((size, size), resample)
    else:
        img = img.resize((size, size), resample)
    return img

In [ ]:
filepaths = glob.glob(f'{DATA_PATH}/test/**/*dcm', recursive=True)
test_df = pd.DataFrame({'path': filepaths, })
test_df['image_id'] = test_df.path.map(
    lambda x: x.split('/')[-1].replace('.dcm', '') 
    + '_image'
)
test_df['study_id'] = test_df.path.map(
    lambda x: x.split('/')[-3].replace('.dcm', '') 
    + '_study'
)
display(test_df.head())
print('test df loaded', test_df.shape)

In [ ]:
counter = 0
images_paths = []
dim_x = []
dim_y = []
os.makedirs(CACHE_PATH, exist_ok=True)
for file in tqdm(test_df.path, desc='cache progress'):
    if file == '':
        counter += 1
    else:
        xray = read_xray(file)
        im = resize(xray, size=IMG_SIZE) # keep_ratio=True to have original aspect ratio
        im.save(CACHE_PATH + '/' + file.split('/')[-1].replace('dcm', 'png'))
        images_paths.append(file.split('/')[-1].replace('dcm', 'png'))
        dim_x.append(xray.shape[1])
        dim_y.append(xray.shape[0])
print('files omitted:', counter)

elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')

In [ ]:
test_df['img'] = images_paths
test_df['dim_x'] = dim_x
test_df['dim_y'] = dim_y
test_df['mc_target'] = 0
test_df['None Opacity'] = 0
display(test_df.head())
print('test df done', test_df.shape)

# 4 classes model

In [ ]:
valid_aug = A.Compose([ToTensorV2(p=1)])

In [ ]:
def flip(img, axis=0):
    if axis == 1:
        return img[::-1, :, :]
    elif axis == 2:
        return img[:, ::-1, :]
    elif axis == 3:
        return img[::-1, ::-1, :]
    elif axis == 4:
        return np.rot90(img, axes=(1, 0))
    elif axis == 5:
        return np.rot90(img, axes=(0, 1))
    else:
        return img
    
class StudyDataset(Dataset):
    
    def __init__(self, df, target='mc_target', 
                 tab_data=None, aug=None, tta=None):
        super().__init__()
        self.df = df 
        self.aug = aug 
        self.target = target
        self.tab_data = tab_data
        self.tta = tta
        
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, idx):
        image = cv2.imread(f'{CACHE_PATH}/{self.df["img"][idx]}')
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  
        mask = np.zeros((image.shape[0], image.shape[1]))
        if self.tta: 
            image = flip(image.copy(), axis=self.tta)
        if self.aug:
            augmented = self.aug(image=image.copy(), mask=mask)
            image = augmented['image'] / 255
            mask = augmented['mask'] / 255
        else:
            image = torch.from_numpy(image).float()
            image = image.permute(2, 1, 0) / 255
            mask = mask / 255
        target = self.df[self.target][idx]
        if self.tab_data:
            tab_data = self.df[['modality','sex','body_part']].values
            tab_data = tab_data[idx]
            tab_data = torch.tensor(tab_data)
            return image, target, mask, tab_data 
        else:
            return image, target, mask
    
dataset_show = StudyDataset(
    df=test_df,
    aug=valid_aug,
    tta=4
)
img_show, lbl_show, mask_show = dataset_show.__getitem__(2)
img_show = img_show.numpy().transpose([1, 2, 0])
img_show = np.clip(img_show, 0, 1)
plt.imshow(img_show)
plt.title('image, target ' + str(lbl_show))
plt.show()
plt.imshow(mask_show)
plt.title('mask, target ' + str(lbl_show))
plt.show()

In [ ]:
class Net(nn.Module):
    
    def __init__(self, params, n_classes=4, tab_flag=False):
        super(Net, self).__init__()
        e = timm.create_model(params['model_arch'], pretrained=False)
        self.b0 = nn.Sequential(
            e.conv_stem,
            e.bn1,
            e.act1
        )
        self.b1 = e.blocks[0]
        self.b2 = e.blocks[1]
        self.b3 = e.blocks[2]
        self.b4 = e.blocks[3]
        self.b5 = e.blocks[4]
        self.b6 = e.blocks[5]
        self.b7 = e.blocks[6]
        self.b8 = nn.Sequential(
            e.conv_head, 
            e.bn2,
            e.act2
        )
        self.final_eff_layer = nn.Linear(e.classifier.in_features, 1000)
        self.mask = nn.Sequential(
            nn.Conv2d(176, 160, kernel_size=3, padding=1),
            nn.BatchNorm2d(160),
            nn.ReLU(inplace=True),
            nn.Conv2d(160, 160, kernel_size=3, padding=1),
            nn.BatchNorm2d(160),
            nn.ReLU(inplace=True),
            nn.Conv2d(160, 1, kernel_size=1, padding=0),
        )
        self.tab_flag = tab_flag
        if self.tab_flag:
            # Tab layers 
            num_features = 3
            self.hidden_size = [10, 6] 
            self.batch_norm1 = nn.BatchNorm1d(num_features)
            self.linear1 = nn.Linear(num_features, self.hidden_size[0])
            self.batch_norm2 = nn.BatchNorm1d(self.hidden_size[0])
            self.linear2 = nn.Linear(self.hidden_size[0], self.hidden_size[1])
            # FINAL LAYER 
            self.final = nn.Linear(1006, n_classes)
        else:
            self.final = nn.Linear(1000, n_classes)

    # @torch.cuda.amp.autocast()
    def forward(self, image, tab_data=None):
        # Image layers
        batch_size = len(image)
        x = 2 * image - 1    
        x = self.b0(x) 
        x = self.b1(x) 
        x = self.b2(x) 
        x = self.b3(x) 
        x = self.b4(x)
        x = self.b5(x) 
        # ============ #
        mask = self.mask(x)
        # ============ #
        x = self.b6(x) 
        x = self.b7(x) 
        x = self.b8(x)
        x = F.adaptive_avg_pool2d(x, 1).reshape(batch_size, -1)
        x = self.final_eff_layer(x)
        if tab_data:
            # TAB layers 
            y = self.batch_norm1(tab_data)
            y = F.relu(self.linear1(y))
            y = self.batch_norm2(y)
            y = F.relu(self.linear2(y))
            # Final Layers 
            x = torch.cat((x, y), dim=1)  # Concatenating image feats + tab feats 
        x = F.relu(x)
        logit = self.final(x)
        return logit, mask

In [ ]:
datasets, loaders = [], []
for tta in TTAS:
    dataset = StudyDataset(
        df=test_df,
        aug=valid_aug,
        tta=tta
    )
    loader = torch.utils.data.DataLoader(
        dataset, 
        batch_size=BATCH_SUBM,
        shuffle=False,
        num_workers=WORKERS
    )
    datasets.append(dataset)
    loaders.append(loader)

In [ ]:
models = []
for ver, folds in MDLS_FOLDS.items():
    with open(f'{MDLS_PATHS[ver]}/params.json') as file:
        params = json.load(file)
    print('version:', ver, '| loaded params:', params, '\n')
    for fold in folds:
        model = Net(params) 
        path = f'{MDLS_PATHS[ver]}/model_best_fold_{fold}.pth'
        state_dict = torch.load(path, map_location=torch.device('cpu'))
        model.load_state_dict(state_dict)
        model.float()
        model.eval()
        model.cuda(DEVICE)
        models.append(model)
        print('loaded:', path)
    del state_dict, model; gc.collect()

In [ ]:
preds = []
with torch.no_grad():
    for i, model in enumerate(models):
        for j, loader in enumerate(loaders):
            preds_tta = []
            for data in tqdm(loader, desc=f'model {i}, loader {j}'):
                img_data = data[0]
                img_data = img_data.to(DEVICE)
                logits, _ = model(img_data)
                probs = torch.softmax(logits, 1)
                probs = np.squeeze(probs.cpu().numpy())
                preds_tta.extend(probs.tolist())
            preds.append(preds_tta)
preds = np.mean(preds, axis=0)

elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')

In [ ]:
name2fname = {
    'Negative for Pneumonia': 'negative', 
    'Typical Appearance': 'typical', 
    'Indeterminate Appearance': 'indeterminate', 
    'Atypical Appearance': 'atypical'
    
}
name2label = {v: i for i, (k, v) in enumerate(name2fname.items())}
print(name2label)
label2name  = {v:k for k, v in name2label.items()}
print(label2name)
cols_classes = [str(x) for x in list(name2label.values())]
for i, col in enumerate(cols_classes):
    test_df[col] = preds[:, i]
display(test_df.head())
print(test_df.shape)

# 2 classes model

In [ ]:
models = []
for ver, folds in MDLS_FOLDS_TWOCLS.items():
    with open(f'{MDLS_PATHS_TWOCLS[ver]}/params.json') as file:
        params = json.load(file)
    print('version:', ver, '| loaded params:', params, '\n')
    for fold in folds:
        model = Net(params, n_classes=1) 
        path = f'{MDLS_PATHS_TWOCLS[ver]}/model_best_fold_{fold}.pth'
        state_dict = torch.load(path, map_location=torch.device('cpu'))
        model.load_state_dict(state_dict)
        model.float()
        model.eval()
        model.cuda(DEVICE)
        models.append(model)
        print('loaded:', path)
    del state_dict, model; gc.collect()

In [ ]:
preds = []
with torch.no_grad():
    for i, model in enumerate(models):
        for j, loader in enumerate(loaders):
            preds_tta = []
            for data in tqdm(loader, desc=f'model {i}, loader {j}'):
                img_data = data[0]
                img_data = img_data.to(DEVICE)
                logits, _ = model(img_data)
                probs = torch.sigmoid(logits)
                probs = np.squeeze(probs.cpu().numpy())
                preds_tta.extend(probs.tolist())
            preds.append(preds_tta)
preds = np.mean(preds, axis=0)

elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')

In [ ]:
test_df.loc[:, 'None Opacity'] = preds
display(test_df.head())
print(test_df.shape)

# MMDet model infer

In [ ]:
import mmcv
import mmdet
from ensemble_boxes import *
from mmdet.apis import init_detector, inference_detector
print(mmdet.__version__)

In [ ]:
BXS = True
VER_BXS = 'v0'
if KAGGLE:
    MDLS_BXS_PATH = f'../input/siim-mmdetection-train-demo' 
else:
    MDLS_BXS_PATH = f'/u01/mrorange/siim/models_mmdet_{VER_BXS}'
TH = .35
IOU_TH = .6

In [ ]:
with open(f'{MDLS_BXS_PATH}/params.json') as file:
    params_bxs = json.load(file)
print(params_bxs)

In [ ]:
checkpoint = f'{MDLS_BXS_PATH}/epoch_6.pth'
cfg = f'{MDLS_BXS_PATH}/init_config.py'
model_bxs = init_detector(cfg, checkpoint, device='cuda:0')

In [ ]:
def plot_imgs(imgs, cols=2, size=10, is_rgb=True, title='', cmap='gray', img_size=None):
    rows = len(imgs) // cols + 1
    fig = plt.figure(figsize=(cols * size, rows * size))
    for i, img in enumerate(imgs):
        if img_size is not None:
            img = cv2.resize(img, img_size)
        fig.add_subplot(rows, cols, i + 1)
        plt.axis('off')
        plt.imshow(img, cmap=cmap)
    plt.suptitle(title)
    plt.axis('off')
    
def draw_bbox(img, box, label, color, thickness=3):   
    alpha = .1
    alpha_box = .4
    overlay_bbox = img.copy()
    overlay_text = img.copy()
    output = img.copy()
    text_width, text_height = cv2.getTextSize(label.upper(), cv2.FONT_HERSHEY_SIMPLEX, .6, 1)[0]
    cv2.rectangle(overlay_bbox, 
                  (box[0], box[1]), 
                  (box[2], box[3]), 
                  color, -1)
    cv2.addWeighted(overlay_bbox, alpha, output, 1 - alpha, 0, output)
    cv2.rectangle(overlay_text, 
                  (box[0], box[1] - 7 - text_height), 
                  (box[0] + text_width + 2, box[1]),
                  (0, 0, 0), -1)
    cv2.addWeighted(overlay_text, alpha_box, output, 1 - alpha_box, 0, output)
    cv2.rectangle(output, 
                  (box[0], box[1]), 
                  (box[2], box[3]),
                  color, thickness)
    cv2.putText(output, 
                label.upper(), 
                (box[0], box[1]-5),
                cv2.FONT_HERSHEY_SIMPLEX, 
                .6, (255, 255, 255), 1, 
                cv2.LINE_AA)
    return output

In [ ]:
imgs_idxs = test_df.img.values
imgs = []
preds_bxs = []
for i, img_name in enumerate(tqdm(imgs_idxs, desc=f'test {ver}')):
    ratio_x = IMG_SIZE / test_df.loc[test_df['img'] == img_name, 'dim_x'].values[0]
    ratio_y = IMG_SIZE / test_df.loc[test_df['img'] == img_name, 'dim_y'].values[0]
    img = cv2.imread(f'{CACHE_PATH}/{img_name}')
    result = inference_detector(model_bxs, img)
    boxes_list = [list(x[:, :4] / IMG_SIZE) for x in result if x.shape[0] != 0]
    boxes_list =  [item for sublist in boxes_list for item in sublist]
    scores_list = [x[:, 4].tolist() for x in result if x.shape[0] != 0]
    scores_list =  [item for sublist in scores_list for item in sublist]
    labels_list = [[i] * x.shape[0] for i, x in enumerate(result) if x.shape[0] != 0]
    labels_list =  [item for sublist in labels_list for item in sublist]
    boxes, scores, box_labels = nms(
        boxes=[boxes_list], 
        scores=[scores_list], 
        labels=[labels_list], 
        weights=None,
        iou_thr=IOU_TH
    )
    boxes *= IMG_SIZE
    if i <= 3:
        for label_id, box, score in zip(box_labels, boxes, scores):
            if score >= TH:
                color = [255, 255, 255]
                img = draw_bbox(
                    img, 
                    list(np.int_(box)), 
                    'predict', 
                    color
                )
        imgs.append(img)
    string = ''
    for label_id, box, score in zip(box_labels, boxes, scores):
        if score >= TH:
            str_boxes = ' '.join([
                str(int(box[0] / ratio_x)),
                str(int(box[1] / ratio_y)),
                str(int(box[2] / ratio_x)),
                str(int(box[3] / ratio_y))
            ])
            string += f'opacity {score:0.2f} {str_boxes} '
    string = string.strip()
    preds_bxs.append(string if string else 'none 1 0 0 1 1')
plot_imgs(imgs, size=8, cols=4, cmap=None)

elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')

# Post-processing for submission

In [ ]:
if BXS:
    image_df = pd.DataFrame({
        'image_id': test_df.image_id.tolist(),
        'PredictionString_img': preds_bxs
    })
else:
    image_df = pd.DataFrame({
        'image_id': test_df.image_id.tolist(),
        'PredictionString_img': ["none 1 0 0 1 1"] * len(test_df.image_id.tolist())
    })
display(image_df.head())
print('image df done', image_df.shape)

In [ ]:
image_df[image_df['image_id'] == '1b282faf0f42_image']

In [ ]:
def get_predstring(row, thr=0):
    string = ''
    for idx in range(4):
        conf =  row[str(idx)]
        if conf > thr:
            string += f'{label2name[idx]} {conf:0.2f} 0 0 1 1 '
    string = string.strip()
    return string

def get_prednoneop(row):
    string = row['PredictionString_img']
    if string != 'none 1 0 0 1 1':
        string += f' none {row["None Opacity"]:0.2f} 0 0 1 1'
    string = string.strip()
    return string

cols = ['image_id', 'study_id']
print('test shape:', test_df.shape)
subm_df = pd.merge(test_df[cols], image_df, 
                   left_on='image_id', right_on='image_id', 
                   how='left')
subm_df = pd.merge(subm_df, 
                   test_df.groupby(['study_id'])[cols_classes].mean().reset_index(), 
                   left_on='study_id', right_on='study_id', 
                   how='left')
subm_df = pd.merge(subm_df, 
                   test_df.groupby(['study_id'])['None Opacity'].mean().reset_index(), 
                   left_on='study_id', right_on='study_id', 
                   how='left')
subm_df['PredictionString_sty'] = subm_df.apply(get_predstring, axis=1)
subm_df['PredictionString_img'] = subm_df.apply(get_prednoneop, axis=1)
display(subm_df.head())
print('unique study ids:', len(subm_df.study_id.unique()))
print('unique image ids:', len(subm_df.image_id.unique()))
print('subm shape:', subm_df.shape)

In [ ]:
new_cols = {'image_id': 'img', 'study_id': 'id'}
classes = [str(x) for x in list(name2fname.keys())]
new_cols.update({k: v for k, v in zip(cols_classes, classes)})
pseudo_df = subm_df[
    ['image_id', 'study_id'] + cols_classes + ['None Opacity']
].rename(columns=new_cols).drop_duplicates()
pseudo_df.to_csv('pseudo_study_level.csv', index=False)
display(pseudo_df.head())
print('submission done:', pseudo_df.shape)

In [ ]:
subm_df = pd.concat([
    subm_df[['image_id', 'PredictionString_img']].rename(
        columns={'image_id': 'id', 'PredictionString_img': 'PredictionString'}
    ).drop_duplicates(), 
    subm_df[['study_id', 'PredictionString_sty']].rename(
        columns={'study_id': 'id', 'PredictionString_sty': 'PredictionString'}
    ).drop_duplicates()
])
subm_df.to_csv('submission.csv', index=False)
display(subm_df.head())
print('submission done:', subm_df.shape)

In [ ]:
shutil.rmtree(CACHE_PATH)
print(f'{CACHE_PATH} -> cache deleted')

elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')